In [2]:
import numpy as np

# forward backward algorithm given the parameters for the HMM
def forward_backward(y, P, B, pi):
    """
    y: observation ids of the obs states of len T
    P: transition matrix of (S, S) where S is the # hidden states
    B: observation transitions matrix for size (S, K), where K is the # observation states
    pi: initial distribution over S states
    """

    S, K = B.shape
    T = len(y)

    # forward probs
    f = np.zeros((T+1, S))
    f_cs = np.zeros(T)
    f_hat = np.ones_like(f)
    f[0,:] = pi
    for t in range(1,T+1):  # 1, 2, ..., T
        f[t,:] = f[t-1,:] @ P @ np.diag(B[:, y[t-1]])
        f_cs[t-1] = np.sum(f[t,:])
        f_hat[t,:] = f[t,:] / f_cs[t-1]

    # backward probs
    b = np.zeros((T+1, S))
    b_cs = np.zeros(T)
    b_hat = np.ones_like(b)
    b[T,:] = np.ones(S)
    for t in range(T, 0, -1):  # T, T-1, ..., 1
        b[t-1,:] = P @ np.diag(B[:, y[t-1]]) @ b[t,:]
        b_cs[t-1] = np.sum(b[t-1,:])
        b_hat[t-1,:] = b[t-1,:] / b_cs[t-1] 

    post_probs = np.zeros((T+1, S))

    for t in range(T+1):
        post_probs[t] = f[t] * b[t] 
        normalizations = np.sum(f[t] * b[t])
        post_probs[t] /= normalizations
    return f_hat, b_hat, post_probs


def BW_step(y, P0, B0, pi0):

    # E-step

    f_hat, b_hat, post_probs = forward_backward(y=y, P=P0, B=B0, pi=pi0)

    T = len(y)
    S, K = B0.shape

    # M-step

    # transition_mat = np.zeros((T+1,S,S))

    f_hat_vec = np.expand_dims(f_hat[:T], axis=-1)
    b_hat_vec = np.expand_dims(b_hat[1:], axis=-2)
    # P0_mat = np.expand_dims(P0, axis=0)

    print('f_hat_vec = ', f_hat_vec.shape)
    print('b_hat_vec = ', b_hat_vec.shape)

    # print('P0mat = ', P0_mat.shape)


    obs_mat = np.zeros((T,1,S))
    for t in range(T):
        obs_mat[t,0,:] = B0[:, y[t]]

    
    trans = (f_hat_vec @ (b_hat_vec * obs_mat)) * P0 

    print('trans = ', trans)

    normalizations = np.expand_dims(np.sum(trans, axis=(-1,-2)), axis=(-1,-2))

    print('trans = ', trans.shape)

    print('norms = ', normalizations.shape)

    trans_probs = trans / normalizations

    for t in range(T):
        print(f'trans_probs[{t}] = ', trans_probs[t])


    pi = post_probs[0]
    P = np.sum(trans_probs, axis=0) / np.sum(post_probs, axis=0)

    print('new P = ', P)


# create a HMM instance and a set for observations
P = np.array([[0.7,0.3],[0.3,0.7]])
B = np.array([[0.9,0.1],[0.2,0.8]])
y = np.array([0,0,1,0,0])
pi = np.array([0.5,0.5])
f_hat, b_hat, post_probs = forward_backward(y, P, B, pi)  # run the algorithm to estimate the posterior prob at each state seperately

# print
for t in range(0, len(y)):
    print(f'f_hat[{t},:] = ', f_hat[t])
for t in range(len(y)+1):
    print(f'b_hat[{t},:] = ', b_hat[t])
for t in range(len(y)+1):
    print(f'post_probs[{t},:] = ', post_probs[t])

# P0 = np.array([[0.5,0.5],[0.3,0.7]])
# B0 = np.array([[0.3,0.7],[0.8,0.2]])
# pi0 = np.array([1,0])

# y0 = np.array([0,0,0,0,0,1,1,0,0,0])


# BW_step(y0, P0, B0, pi0)


        

    



































f_hat[1,:] =  [0.81818182 0.18181818]
f_hat[2,:] =  [0.88335704 0.11664296]
f_hat[3,:] =  [0.19066794 0.80933206]
f_hat[4,:] =  [0.730794 0.269206]
f_hat[5,:] =  [0.86733889 0.13266111]
b_hat[0,:] =  [0.64693556 0.35306444]
b_hat[1,:] =  [0.5923176 0.4076824]
b_hat[2,:] =  [0.37626718 0.62373282]
b_hat[3,:] =  [0.65334282 0.34665718]
b_hat[4,:] =  [0.62727273 0.37272727]
b_hat[5,:] =  [1. 1.]
post_probs[0,:] =  [0.64693556 0.35306444]
post_probs[1,:] =  [0.86733889 0.13266111]
post_probs[2,:] =  [0.82041905 0.17958095]
post_probs[3,:] =  [0.30748358 0.69251642]
post_probs[4,:] =  [0.82041905 0.17958095]
post_probs[5,:] =  [0.86733889 0.13266111]
